## 模型持久化文件介绍


### 保存方式一

#### 保存代码：
```
saver.save(sess,'./tmp/my_model/model.ckpt')
```

计算图结构（Meta graph）：
model.ckpt-30001.meta 

最新版本模型和所有版本模型路径：
checkpoint 

model.ckpt文件（Checkpoint file），保存所有的weights,biases,gradients和其他variables的值：

model.ckpt-30001.data-00000-of-00001
model.ckpt-30001.index


```
#恢复模型
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('path/to/model/model.meta')
    new_saver.restore(sess,tf.train.latest_checkpoint('path/to/model/model.ckpt‘))
    
    #加载部分变量
    graph = tf.get_default_graph()
    fc7= graph.get_tensor_by_name('fc7:0')
```
**此种保存形式只能在Tensorflow框架下使用**

### 保存方式二

.pb格式文件

#### 保存代码:
```
from tensorflow.python.framework import graph_util

# convert_variables_to_constants 需要指定output_node_names，list()，可以多个
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['op_to_store'])

    # 测试 OP
    feed_dict = {x: 10, y: 3}
    print(sess.run(op, feed_dict))

    # 写入序列化的 PB 文件
    with tf.gfile.FastGFile(pb_file_path+'model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())
```

#### 恢复模型：

```
#读取google训练好的模型classify_image_graph_def.pb
inception_graph_def_file=os.path.join(LOG_DIR,'classify_image_graph_def.pb')
with tf.Session() as sess:
    #创建一个图来存放训练好的模型
    with tf.gfile.FastGFile(inception_graph_def_file,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name='')
        
    #保存图结构：保存之前将之前保存的events.out.tfevents......iMac文件删除
    #tools.delete_dir_file(LOG_DIR,'iMac')
    
    writer=tf.summary.FileWriter(LOG_DIR,sess.graph)
    writer.close()

```

### [点击下载inception-v3.pd文件](http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz)
数据解压之后有以下文件：

* classify_image_graph_def.pb
* imagenet_2012_challenge_label_map_proto.pbtxt
* imagenet_synset_to_human_label_map.txt

其中后面两个用于最后的分类显示，相当于网络最后一层softmax得出得分类是一个字符串，然后这个字符串在文件里面查找出对应的human_label.直接用inception-v3分类需要

In [ ]:

#现在下载好inception-v3.pd文件，将其读入并保存计算图，
#以便与使用tensorboard查看其结构
#读取google训练好的模型classify_image_graph_def.pb

import tensorflow as tf 

INCEPTION_V3_PD='tmp/inception_v3/classify_image_graph_def.pb'
INCEPTION_V3_GRAPH_DEF='tmp/inception_v3_graph_def'

with tf.Session() as sess:
    #创建一个图来存放训练好的模型
    with tf.gfile.FastGFile(INCEPTION_V3_PD,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name='inception-v3')
    
    writer=tf.summary.FileWriter(INCEPTION_V3_GRAPH_DEF,sess.graph)
    writer.close()


运行后得到文件：

events.out.tfevents.1530084131.c82682dd8be5

执行下面命令便可查看模型结构：

tensorboard --logdir=path/to/events.out.tfevents..._save_dir

输入张量
![](https://upload-images.jianshu.io/upload_images/1271438-12159ef39308d192.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

![](https://upload-images.jianshu.io/upload_images/1271438-a1fd3f34e2b40aa9.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

# 接下来将进行的操作便是：

使用inception-v3，提取我们的测试图片特征，获得pool_3_reshape输出1x2048
得到的数据来训练我们构建的全连接网络，以此实现自定义分类


In [ ]:
#获取数据

import os
import glob
import numpy as np

FLOWER_PHOTOS_PATH='flower_photos'
TEST_ACCOUNT=1000


def get_datasets():
    sub_dirs=[_[0] for _ in os.walk(FLOWER_PHOTOS_PATH)][1:]
    images=[]
    
    """
    flower_photos/daisy 0
    flower_photos/dandelion 1
    flower_photos/roses 2
    flower_photos/sunflowers 3
    flower_photos/tulips 4
    """
    labels=[]
    
    for index,sub_dir in enumerate(sub_dirs):
        file_names=glob.glob(sub_dir+'/*.jpg')
        images.extend(file_names)
        labels.extend(np.full(len(file_names),index))
          
    #乱序
    state=np.random.get_state()
    np.random.shuffle(images)
    np.random.set_state(state)
    np.random.shuffle(labels)
    
    """
    ['flower_photos/tulips/3909355648_42cb3a5e09_n.jpg', 
    'flower_photos/tulips/5634767665_0ae724774d.jpg', 
    'flower_photos/dandelion/17482158576_86c5ebc2f8.jpg', 
    'flower_photos/tulips/16265876844_0a149c4f76.jpg',
    'flower_photos/dandelion/344318990_7be3fb0a7d.jpg']
    """
    return images,labels
    
images,labels=get_datasets()
images[:5],labels[:5]

In [ ]:
#将数据转化成自定义全连接网络可用数据

NUM_CLASS=5
OUTPUT_FILE='flower_photos/flower_processed.npy'

from tensorflow.python.platform import gfile
import tensorflow as tf
import numpy as np

one_hot=np.eye(NUM_CLASS)
labels_one_hot=np.apply_along_axis(lambda x:one_hot[x],0,labels)


DECODE_JPEG_CONTENTS='DecodeJpeg/contents:0'
POOL_3_RESHAPE_NAME='pool_3/_reshape:0'


#读取inception-v3.pd
INCEPTION_V3_PD='tmp/inception_v3/classify_image_graph_def.pb'

with tf.gfile.FastGFile(INCEPTION_V3_PD,'rb') as f:
    graph_def=tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
    decode_jpeg_contents_tensor,pool_3_reshape_tensor=tf.import_graph_def(
        graph_def,
        return_elements=[DECODE_JPEG_CONTENTS,POOL_3_RESHAPE_NAME]
    )

decode_jpeg_contents_tensor,pool_3_reshape_tensor
    
def get_pool3_reshape_values(sess,pool3_reshape_tensor,image_path):
    image_raw_data=gfile.FastGFile(image_path,'rb').read()
#     image_data=tf.image.decode_jpeg(image_raw_data)
    pool3_reshape_value=sess.run(pool3_reshape_tensor,feed_dict={
        DECODE_JPEG_CONTENTS:image_raw_data
    })
    return pool3_reshape_value
    

with tf.Session() as sess:
    images_2048=[]
    for path in images:
        img=get_pool3_reshape_values(sess,pool_3_reshape_tensor,path)
        images_2048.append(img)
    
    processed_data=np.array([images_2048,labels_one_hot])
    np.save(OUTPUT_FILE,processed_data)
    
print images_2048[:5],labels_one_hot[:5]

In [ ]:

#迁移网络结构-向前传播-自定义部分
import tensorflow as tf

POOL_3_RESHAPE_NODE=2048
MIDDLE_NODE=500
LEARNING_RATE=0.0001
STEPS=300
BATCH=32


def inference_custom(input_tensor):
    
    #输入为pool3_reshape_tensor,shape:[1,2048]
    
    #自定义全连接层一
    with tf.variable_scope('custom_layer_1'):
        cl1_weights=tf.get_variable(
            'weights',
            [2048,MIDDLE_NODE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        
        cl1_biases=tf.get_variable(
            'biases',
            [MIDDLE_NODE],
            initializer=tf.constant_initializer(0.1)
        )
    
        cl1=tf.nn.relu(tf.matmul(input_tensor,cl1_weights)+cl1_biases)
    
    #自定义全连接层二
    with tf.variable_scope('custom_layer_2'):
        cl2_weights=tf.get_variable(
            'weights',
            [MIDDLE_NODE,NUM_CLASS],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        
        cl2_biases=tf.get_variable(
            'biases',
            [NUM_CLASS],
            initializer=tf.constant_initializer(0.1)
        )
    
        logit=tf.matmul(cl1,cl2_weights)+cl2_biases
    
    return logit



In [ ]:
IMAGES,LABELS=np.load(OUTPUT_FILE)

TRAIN_IMAGES,TRAIN_LABELS=IMAGES[TEST_ACCOUNT:],LABELS[TEST_ACCOUNT:]
TEST_IMAGES,TEST_LABELS=IMAGES[:TEST_ACCOUNT],LABELS[:TEST_ACCOUNT]

def get_batch(batch,train=True):
    if train:
        state=np.random.get_state()
        images=np.random.choice(TRAIN_IMAGES,size=batch)
        np.random.set_state(state)
        labels=np.random.choice(TRAIN_LABELS,size=batch)
    else:
        images,labels=TEST_IMAGES,TEST_LABELS
    return images,labels

print get_batch(2)

In [ ]:
#迁移网络结构-训练部分


from tensorflow.python.platform import gfile
MODE_SAVE_PATH='tmp/transer/model.ckpt'


def train():

    #输入变量初始化
    x=tf.placeholder(tf.float32,[None,POOL_3_RESHAPE_NODE])
    y_=tf.placeholder(tf.float32,[None,NUM_CLASS],name='y-input')
    
    #向前传播
    y=inference_custom(x)
    global_step=tf.Variable(0,trainable=False)

    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y,
        labels=tf.argmax(y_,1)
    )
    
    loss=tf.reduce_mean(cross_entropy)
    train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss,global_step=global_step)
    
    saver=tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(STEPS):
            xs,ys=get_batch(BATCH)
            loss,step=sess.run([loss,global_step],feed_dict={
                x:xs,
                y_:ys   
            })
            
            if i% 20==0:
                print 'Step:%d, loss=%f' % (step,loss)
            
            #保存模型
            saver.save(sess,MODE_SAVE_PATH,global_step=step)

def main(argv=None):
    train()
    
if __name__=='__main__':
    tf.get_variable_scope().reuse_variables()
    tf.app.run()